In [ ]:
import os
from voc12 import my_dataloader
from utility import image_util, util
from torch.utils.data import DataLoader
import torch
from misc import imutils
import importlib
import torch.nn.functional as F
import numpy as np

In [ ]:
img_path = "voc12/val.txt"
voc_root = "../Dataset/VOC2012/"
cam_root = "../irn_result/cam/"

In [ ]:
dataset = my_dataloader.VOC12_CropClassificationDatasetMSF(img_path, voc_root, cam_root, scales = (1.0, 1.5, 0.5, 2.0))

In [ ]:
data_loader = DataLoader(dataset)

In [ ]:
data = next(iter(data_loader))

In [ ]:
msf = data['msf_img_list']
crop_boxes = data['crop_boxes']
crop_labels = data['crop_labels']
cams = data['cams']

In [ ]:
crop_boxes[0]

In [ ]:
model = getattr(importlib.import_module("net.resnet50_cam"), 'CAM')()

model.load_state_dict(torch.load("../sess/voc_sess/resnet50_crop_cam" + '.pth'), strict=True)
        
model.eval()
model = model.to('cuda:0')
device = 'cuda:0'

In [ ]:
img_name = data['name']
msf = data['msf_img_list']
org_size = data['img'].shape[1:]

crop_labels = data['crop_labels']
crop_boxes = data['crop_boxes']

cam_list = []

key = torch.sum(torch.cat(crop_labels, dim=0), dim=0)
key = torch.nonzero(key)[:, 0]
with torch.no_grad():
    for idx, msf_img in enumerate(msf):
        
        msf_img = msf[idx]
        label = crop_labels[idx][0]
        
        size = (crop_boxes[idx][3] - crop_boxes[idx][1], crop_boxes[idx][2] - crop_boxes[idx][0])
        
        strided_up_size = imutils.get_strided_up_size(size, 16)
        
        valid_cat = torch.nonzero(label)[:, 0]
        
        outputs = [model(img[0].to(device)) for img in msf_img]
        
        highres_cam = [F.interpolate(torch.unsqueeze(o, 1), strided_up_size,
                                    mode='bilinear', align_corners=False) for o in outputs]

        
        highres_cam = torch.sum(torch.stack(highres_cam, 0), 0)[:, 0, :size[0], :size[1]]
        
        highres_cam = highres_cam[valid_cat]
        highres_cam /= F.adaptive_max_pool2d(highres_cam, (1, 1)) + 1e-5
        highres_cam = highres_cam[0].cpu().numpy()
        highres_cam = image_util.crop_cam_to_org_cam(highres_cam, crop_boxes[idx], org_size)
        
        cam_list.append(highres_cam)
        
    cam_stack = np.stack(cam_list)

        
        
        

In [ ]:
np_cams = np.array(cams[0])
np_cams.shape

image_util.image_show(cam_stack[1], np_cams[1], idx=(1,2))


In [ ]:
np.save("temp.npy", {'keys' : key, 'cams':cam_stack})

In [ ]:
print(img_name)